In [1]:
import plotly.express as px
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import math 

#import csv
pd.set_option('display.max_rows', None)  # Display all rows

In [2]:
if not os.path.exists("images"):
    os.mkdir("images")

day = "07"
month = "09"
date = day + "_" + month #change date accordingly
date_year = "2023-" + month + "-" + day

def process_file(date):
    file_path = os.path.join('data_fabrica/',f'task_{date}.csv')
    print(file_path)
    if os.path.exists(file_path):

        df = pd.read_csv(file_path, parse_dates=['ts'])

        # Remove first empty column if it exists
        if df.shape[1] == 5:
            df = df.drop(df.columns[0], axis=1)
        # Remove entries with 'sec' equal to 0
        df= df[df['sec'] != 0]

        # Write the updated DataFrame back to the CSV file
        df.to_csv(file_path, index = False)
        # Reset the index of the DataFrame
        df = df.reset_index(drop = True)

        # Create a new column 'task_label' based on the mapping
        task_label_mapping = {0: 'Break', 1: 'TEKOX Red Black Cables', 2: 'TEKOX Blue Brown Cables', 3: 'MFALG Assembly',
                            4: 'Screws in Boards', 5: 'Extra activity 1', 6: 'Extra Activity 2', 7: 'Unkown Activity'}
        df['task_name'] = df['task'].map(task_label_mapping)


        emp_label_mapping = {0: 'Fernando', 1: 'Antonio', 2: 'Daniela', 3: 'Diogo', 4: 'Elisabete',5: 'Isabel', 6: 'Joao', 
                            7:'Joaquim',8:'Lidia',9:'Maria',10:'Mario',11:'Patricia',12:'Raquel',13:'Ana',14:'Paulo',15:'Luis'}
        df['emp_name'] = df['empid'].map(emp_label_mapping)
        #display(df)
    else:
        return pd.DataFrame
    return df

df = process_file(date)
#display(df)

data_fabrica/task_07_09.csv


In [3]:
def remove_outliers(df,alpha):
    ref_av_time = np.array([-1,12.0, 12, 144, 40, 60, -1, -1])

    ref_av_time[0] = df[df['task'] == 0]['sec'].mean()
    #ref_av_time[5] = df[df['task'] == 5]['sec'].mean()
    ref_av_time[6] = df[df['task'] == 6]['sec'].mean()
    ref_av_time[7] = df[df['task'] == 7]['sec'].mean()

    my_med_time = np.zeros(8)
    my_av_time = np.zeros(8)
    for task_id in range(1,8):
        my_med_time[task_id] = df[df['task'] == task_id]['sec'].median()
        my_av_time[task_id] = df[df['task'] == task_id]['sec'].mean()
    print(my_med_time)
    
    print("Reference average")
    print(ref_av_time)
    outliers = 0
    use_ref = 0
    print("Outliers:")
    for index, row in df.iterrows():
        for task_id in range(1,8):
                if ((row['task'] == task_id) and ((row['sec'] < (1 - alpha) * my_med_time[task_id]) or (row['sec'] > (2) * my_med_time[task_id]))):
                    outliers += 1
                    #print(row['task_name'],row['sec'])
                    df = df.drop(index=index)
    
    mean_and_variance(df)
    print(f"Outliers removed: {outliers}")
    return df

def mean_and_variance(df):

    av_time = np.zeros(8)
    var_time = np.zeros(8)
    med_time = np.zeros(8)
    count_task = np.zeros(8)
    for task_id in range(0,8):
        count_task[task_id] = df.loc[df['task'] == task_id].shape[0]
        task_data = df[df['task'] == task_id]['sec']
        sum_task = task_data.sum()
        av_time[task_id] = task_data.mean()
        med_time[task_id] = task_data.median()
        var_time[task_id] = task_data.var()
        
        if not math.isnan(av_time[task_id]):
            print(f"Task: {task_id}")
            print(f"Average: {round(av_time[task_id],2)}")
            print(f"Median: {round(med_time[task_id],2)}")
        #if not math.isnan(var_time[task_id]):
            print(f"Variance: {round(var_time[task_id],2)}")        
            std_dev = np.sqrt(var_time[task_id])
            print(f"Standard deviation: {round(std_dev,2)}")
            print(f"Count: {count_task[task_id]}")
            print(f"total secs: {sum_task}")
            #print(f"mean i guess:  {sum_task/count_task[task_id]}")
            count_per_hour = 3600*count_task[task_id]/sum_task
            print(f"Count per hour: {round(count_per_hour)}")
            print()
    return 

if not df.empty:
    df1 = remove_outliers(df, alpha = 0.5)
#display(df1)

[ 0. nan nan nan nan 24. nan nan]
Reference average
[1136.66666667   12.           12.          144.           40.
   60.                   nan           nan]
Outliers:
Task: 0
Average: 1136.67
Median: 80.0
Variance: 3452097.33
Standard deviation: 1857.98
Count: 3.0
total secs: 3410
Count per hour: 3

Task: 5
Average: 24.41
Median: 24.0
Variance: 23.55
Standard deviation: 4.85
Count: 444.0
total secs: 10838
Count per hour: 147

Outliers removed: 38


In [4]:
#taskdistance_01_08.csv
distance_near = 800
distance_delta = 50


def process_file_distance(date):
    file_path = os.path.join('data_distance/',f'taskdistance_{date}.csv')
    if os.path.exists(file_path):

        df = pd.read_csv(file_path, parse_dates=['ts'])
        #print(df.shape[1] == 3)
        #print(not('state' in df.columns))
        # Remove first empty column if it exists
        if ((df.shape[1] == 3) & (not ('state' in df.columns) | (not ('state1' in df.columns)))):
            df = df.drop(df.columns[0], axis=1)

            
        # Write the updated DataFrame back to the CSV file
        df.to_csv(file_path, index = False)
        # Condition 1: If distance is less than 800, set state to 1 (near)
        df['state'] = (df['distance'] < distance_near).astype(int)

        # Condition 2: If delta of distance is less than 5cm, set state to 0 (away)
        condition = abs(df['distance'].diff()) < distance_delta
        df.loc[condition, 'state'] = 0

        # Condition 3: If state is near away near or away near away, change the state of the middle to match the others
        near_away_near = (df['state'] == 1) & (df['state'].shift() == 0) & (df['state'].shift(-1) == 0)
        away_near_away = (df['state'] == 0) & (df['state'].shift() == 1) & (df['state'].shift(-1) == 1)
        condition2 = near_away_near | away_near_away
        df.loc[condition2, 'state'] = df['state'].shift(-1)
        df.loc[condition2, 'state'] = df['state'].shift(-1)
        
        # Write the updated DataFrame back to the CSV file
        df.to_csv(file_path, index = False)

        # Reset the index of the DataFrame
        df = df.reset_index(drop = True)

    else:
        return pd.DataFrame
    return df
    
def fun_time_near(df):
    
    time_near = time_near = pd.Timedelta(seconds=0)
    near_rows = df[(df['state'] == 1) & (df['state'].shift(-1) == 1)]
    
    if 'time_diff' not in df.columns:
        df['time_diff'] = None  # or initialize with appropriate values

    df.loc[near_rows.index, 'time_diff'] = near_rows['ts'].diff()

    for i in range(len(near_rows) - 1):
        current_index = near_rows.index[i]
        next_index = near_rows.index[i + 1]

        if next_index == current_index + 1:
            time_near += df.loc[next_index, 'time_diff']
            
        else:
            continue

            
    return df, time_near
    
def update_df_distance(df,df_dist):
    # Iterate through the rows of the second DataFrame and update the 'state' in the first DataFrame
    for index, row in df.iterrows():
        # Check conditions
        if row['task'] != 0 and not pd.isna(row['sec']):
            # Find the next timestamp with the same value in df1
            time_range = pd.Timedelta(seconds=30)
            next_ts = df_dist.loc[(df_dist['ts'] > row['ts']) & (df_dist['ts'] <= row['ts'] + time_range), 'ts'].min()

            #print(next_ts)
            # Update 'state' in df1
            if next_ts is not pd.NaT:
                df_dist.loc[df_dist['ts'] == next_ts, 'state'] = 1
    return df_dist

df_dist = process_file_distance(date)
if not df1.empty:
    df_distance = update_df_distance(df1,df_dist)
    df_distance.to_csv(f'data_distance/taskdistance_{date}.csv', index = False)

df_distance_, total_time_near = fun_time_near(df_distance)
print(f"Time person was near: {total_time_near}")
#25 min
#14

Time person was near: 0 days 04:41:10


In [5]:
include_break_pie = 0

def plot_pie(df):

    color_discrete_task = { 'Break': 'navy', 'TEKOX Red Black Cables': 'tomato', 'TEKOX Blue Brown Cables': 'cornflowerblue', 'MFALG Assembly': 'gold',
                      'Screws in Boards': 'darkgray', 'Extra activity 1': 'darkorange', 'Extra Activity 2': 'violet', 'Unkown Activity': 'limegreen'}
    
    color_discrete_emp = {'Fernando': 'cornflowerblue', 'Luis': 'yellowgreen'}

    column_order = ['ts', 'empid','emp_name','task','task_name','sec']
    df = df[column_order]
    # Exclude task '0' from the DataFrame
    if not include_break_pie:
        df1 = df[df['task'] != 0]
    else:
        df1 = df
    #display(df1)
    #TASK
    # Sum the 'sec' values based on the 'task' column
    sum_by_task = df1.groupby('task_name')['sec'].sum().reset_index()

    for index,row in sum_by_task.iterrows():
        print(row['task_name'],row['sec'])
        sum_task_dt =  pd.Timedelta(seconds = row['sec'])
        print(sum_task_dt)

    # Create a pie chart
    fig_task = px.pie(sum_by_task, names='task_name', 
                      values='sec',
                      color = 'task_name',
                      color_discrete_map = color_discrete_task, 
                      width = 600)
                      #title='Time Spent in Each Task')
    fig_task.show()
    fig_task.write_image('images/taskpie_' + date + '.png')



    #EMPLOYEE // do not work in break
    df2 = df1[df1['task'] != 0]
    # Sum the 'sec' values based on the 'emp_name' column
    sum_by_emp = df2.groupby('emp_name')['sec'].sum().reset_index()
    #print(sum_by_emp['sec'])
    #print(sum_by_emp)
    for index,row in sum_by_emp.iterrows():
        print(row['emp_name'],row['sec'])
        sum_emp_dt =  pd.Timedelta(seconds = row['sec'])
        print(sum_emp_dt)

    # Create a pie chart
    fig_emp = px.pie(sum_by_emp, names='emp_name', 
                     values='sec', 
                     color = 'emp_name',
                      color_discrete_map = color_discrete_emp,
                      width = 550)
                      #title='Time Worked by Each Employee')
    fig_emp.show()

    #fig_emp.write_image('images/emppie_' + date + '.png')

    return sum_by_task, sum_emp_dt
sum_by_task, sum_emp_dt = plot_pie(df1)
#for i in unique_values:
#    total_time[i] = 
    

Extra activity 1 10838
0 days 03:00:38


Fernando 10838
0 days 03:00:38


In [6]:
include_break_bar = 0
def plot_bar(date_year,df,df_dist):

    if not include_break_bar:
        df = df[df['task'] != 0]
    else:
        df = df

    color_discrete = { 'Break': 'navy', 'TEKOX Red Black Cables': 'tomato', 'TEKOX Blue Brown Cables': 'cornflowerblue', 'MFALG Assembly': 'gold',
                      'Screws in Boards': 'darkgray', 'Extra activity 1': 'darkorange', 'Extra Activity 2': 'violet', 'Unkown Activity': 'limegreen'}

    # Create bars for the first subplot
    fig = px.bar(df, x="ts", y="sec",
                color="task_name",
                labels={"ts": "Timestamp", "sec": "Duration (seconds)",'task_name': 'Task names'},
                color_discrete_map = color_discrete)


    x = np.array(df_dist['ts'])
    y = np.array(df_dist['state']*24)


    # Get the minimum and maximum values of the y-axis
    y_max = df['sec'].max()

    # Set the y-axis range from the minimum to the maximum + 1
    fig.update_layout(yaxis_range=[0, y_max + 1], xaxis_range = [date_year + ' 11:57:00', date_year + ' 14:40:00'])#y_max + 1])
    fig.update_xaxes(showgrid=True)
    #fig.update_layout(width=700, height = 300) 
    fig.update_layout(width=1050, height = 400) 

    fig.update_traces(width=21000) 
    fig.update_layout(legend=dict( yanchor="top", y=0.99,xanchor="left",
    x=0.80))
    fig.update_layout(bargap=0) # gap between bars of adjacent location coordinates.

    fig.update_layout(font=dict(size=14))
    fig.add_trace(go.Scatter(x=x, y=y, line_shape='hv',line=dict(color='tomato'),name='Presence indicator'))


    fig.show()
    #file_path = "images/27-09-Luis.svg"
    #file_path = "images/31-07-Fernando.svg"
    if date_year == "2023-" + "09" + "-" + "07":
        #file_path = "images/07-09-presence.svg"
        #file_path = "images/31-07-Fernando.pdf"
        file_path = "images/07-09-presence.pdf"
        if os.path.exists(file_path):
            os.remove(file_path)
        fig.write_image(file_path)



plot_bar(date_year,df1, df_dist)

c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [8]:
def plot_line(date_year, df):
    if not include_break_bar:
        df = df[df['task'] != 0]
    else:
        df = df

    color_discrete = {'Break': 'navy', 'TEKOX Red Black Cables': 'tomato', 'TEKOX Blue Brown Cables': 'cornflowerblue',
                      'MFALG Assembly': 'gold', 'Screws in Boards': 'darkgray', 'Extra activity 1': 'darkorange',
                      'Extra Activity 2': 'violet', 'Unkown Activity': 'limegreen'}


    # Group by 'task_name' and 5-minute intervals, and calculate the mean of 'sec'
    freq = '5T'
    df_avg = df.groupby(['task_name', pd.Grouper(key='ts', freq=freq)])['sec'].mean().reset_index()
    df_cumavg = df.groupby(['task_name', pd.Grouper(key='ts', freq=freq)])['sec'].expanding().mean().reset_index()


    # Create a scatter plot for the average values
    fig = px.line(df_avg, x='ts', y='sec', color='task_name',
                    labels={"ts": "Timestamp", "sec": "Average Time (seconds)", 'task_name': 'Task names'},
                    title="Average Time per Task in 5-Minute Intervals",
                    color_discrete_map= color_discrete)
    
    if date_year == "2023-07-31":
        fig.update_layout(yaxis_range = [0,45], xaxis_range=[date_year + ' 14:00:00', date_year + ' 16:30:00'])
    
    elif date_year == "2023-08-01":
        fig.update_layout(yaxis_range = [0,45],  xaxis_range=[date_year + ' 09:00:00', date_year + ' 10:00:00'])
        #fig.update_layout(yaxis_range = [0,150],  xaxis_range=[date_year + ' 16:00:00', date_year + ' 17:45:00'])
    
    elif date_year == "2023-08-02":
        fig.update_layout( xaxis_range=[date_year + ' 09:00:00', date_year + ' 18:00:00'])
    
    elif date_year == "2023-09-06":
        fig.update_layout( xaxis_range=[date_year + ' 09:00:00', date_year + ' 18:00:00'])
    
    elif date_year == "2023-09-07":
        fig.update_layout( xaxis_range=[date_year + ' 09:00:00', date_year + ' 18:00:00'])
    
    elif date_year == "2023-09-27":
        fig.update_layout( xaxis_range=[date_year + ' 09:00:00', date_year + ' 18:00:00'])
    

    fig.show()

plot_line(date_year, df1)

c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [8]:
def discrete_colorscale(bvals, colors):

    bvals = sorted(bvals)
    step = 1/(len(bvals))
    nvals = [round(i*step,2) for i in range(len(bvals)+1)]
    
    dcolorscale = [] #discrete colorscale
    for k in range(len(colors)):
        dcolorscale.extend([[nvals[k], colors[k]], [nvals[k+1], colors[k]]])
    return dcolorscale

def plot_states(date_year,df):
    labels_nr = [0,1,2,3]
    color_discrete = { 'Break': 'navy', 'TEKOX Red Black Cables': 'tomato', 'TEKOX Blue Brown Cables': 'cornflowerblue', 'MFALG Assembly': 'gold',
                      'Screws in Boards': 'darkgray', 'Extra activity 1': 'darkorange', 'Extra Activity 2': 'violet', 'Unkown Activity': 'limegreen'}

    colors = ['tomato', 'cornflowerblue', 'gold' , 'darkgray' ]# '#26828E' , '#35B779', '#6ECE58', '#FDE725']

    dcolorsc = discrete_colorscale(labels_nr, colors)

    step = max(labels_nr)/(len(labels_nr))
    print(step)
    nvals = [round(i*step,2) for i in range(len(labels_nr)+1)]
    print(nvals)
    tickvals = [(float(nvals[k])+float(nvals[k+1]))/2 for k in range(len(nvals)-1)] #position with respect to labels_nr where ticktext is displayed
    print(tickvals)
    ticktext = [f'{labels_nr[k]}' for k in range(0, len(labels_nr))]
    print(ticktext)


    fig = go.Figure(data=go.Heatmap(
            z=df['task'], x=df['ts'], y=df['task'], colorscale=dcolorsc,
            colorbar=dict(
                title="Labels",
                thickness=15,
                tickvals=tickvals,
                ticktext=ticktext
            )
    ))
    fig.update_layout(xaxis_range = [date_year + ' 09:00:00', date_year + ' 18:00:00'])#y_max + 1])

    #fig.update_yaxes(ticktext=labels, tickvals=labels_nr)
    
    # add some time reference lines to the xaxis
    #ticktext = [val.strftime('%Y-%m-%d %H:%M:%S') for val in tickvals]
    #tickvals_start = [val - pd.Timedelta(hours=3) for val in tickvals]
    #ticktext_start = [val.strftime('|<br>%Y-%m-%d<br>%H:%M:%S') for val in tickvals_start]
    #fig.update_xaxes(type="date", tickmode='array', tickvals=tickvals_start, ticktext=ticktext_start)

    labels = ['state','screws']

    fig.update_yaxes(ticktext=labels, tickvals=labels_nr)
    fig.update_xaxes(showgrid=True)


    fig.show()

#plot_states(date_year,df1)

This section will now gather a range of days and present the corresponding graphs


In [9]:
start_date = '31/07/2023'
end_date = '27/09/2023'

start_date = pd.to_datetime(start_date, format='%d/%m/%Y')
end_date = pd.to_datetime(end_date, format='%d/%m/%Y')

# Create a date range
date_range = pd.date_range(start=start_date, end=end_date)

dfs = []
df_all = pd.DataFrame()

def calc_all_time_near(time_near,time_worked):
    
    print(f"Time person was near: {time_near}")
    time_near_seconds = time_near_.total_seconds()

    
    print(f"Time person was using node: {sum_emp_dt}")
    time_used_seconds = sum_emp_dt.total_seconds()


    return time_near_seconds, time_used_seconds

def all_plots(date_year,df,df_distance):
    
    sum_by_task, sum_emp_dt = plot_pie(df)
    plot_bar(date_year,df,df_distance)
    #plot_line(date_year, df)

    return sum_by_task, sum_emp_dt

def plots_all_dates(date_range):
    total_time_near = 0
    total_time_used = 0
    
    for date in date_range:

        day = str(date.day).zfill(2)
        month = str(date.month).zfill(2)
        date_ = day + "_" + month
        print(date_)
        date_year = "2023-" + month + "-" + day

        df = process_file(date_)
        df_dist = process_file_distance(date_)

        if df.empty:
            print("No data in that date")
            continue
                
        if df_dist.empty:
            print("No distance data in that date")
            continue

        df1 = remove_outliers(df, alpha = 0.5)

        df_distance = update_df_distance(df1,df_dist)

        _, sum_emp_dt = all_plots(date_year,df1,df_distance)

        _,time_near_ = fun_time_near(df_distance)

        print(f"Time person was near: {time_near_}")
        time_near_seconds = time_near_.total_seconds()

        
        print(f"Time person was using node: {sum_emp_dt}")
        time_used_seconds = sum_emp_dt.total_seconds()

        #time_near_seconds, time_used_seconds = calc_all_time_near(time_near_,sum_emp_dt)
        #total_time_near += time_near_seconds
        #total_time_used += time_used_seconds
      
        perc_usage = time_used_seconds/time_near_seconds*100
        
        print(f"Percentage of node usage per day: {round(perc_usage,0)} ")
        print()

        print()
        print()
        print()

        dfs.append(df1)

    #perc_total = total_time_used/total_time_near*100
    #print(f"Percentage of total node usage: {round(perc_total,0)} ")


    return dfs


# Prints plot for each date
dfs = plots_all_dates(date_range)
df_all = pd.concat(dfs, ignore_index=True)

# Prits plot for all dates together
_, _ = all_plots(date_year,df_all,df_distance)


31_07
data_fabrica/task_31_07.csv
[ 0. nan nan nan 26. nan nan nan]
Reference average
[580.57142857  12.          12.         144.          40.
  60.                  nan          nan]
Outliers:
Task: 0
Average: 580.57
Median: 166.0
Variance: 1406300.95
Standard deviation: 1185.88
Count: 7.0
total secs: 4064
Count per hour: 6

Task: 4
Average: 27.1
Median: 26.0
Variance: 28.24
Standard deviation: 5.31
Count: 449.0
total secs: 12168
Count per hour: 133

Outliers removed: 20
Screws in Boards 12168
0 days 03:22:48


Fernando 12168
0 days 03:22:48


c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Time person was near: 0 days 03:32:19
Time person was using node: 0 days 03:22:48
Percentage of node usage per day: 96.0 




01_08
data_fabrica/task_01_08.csv
[  0.  nan  nan 112.  25.  nan  nan  nan]
Reference average
[467.33333333  12.          12.         144.          40.
  60.                  nan          nan]
Outliers:
Task: 0
Average: 467.33
Median: 292.0
Variance: 303956.33
Standard deviation: 551.32
Count: 3.0
total secs: 1402
Count per hour: 8

Task: 3
Average: 117.56
Median: 112.0
Variance: 281.53
Standard deviation: 16.78
Count: 45.0
total secs: 5290
Count per hour: 31

Task: 4
Average: 26.58
Median: 25.0
Variance: 24.83
Standard deviation: 4.98
Count: 90.0
total secs: 2392
Count per hour: 135

Outliers removed: 2
MFALG Assembly 5290
0 days 01:28:10
Screws in Boards 2392
0 days 00:39:52


Fernando 7682
0 days 02:08:02


...

Luis 1478
0 days 00:24:38


c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Time person was near: 0 days 01:26:38
Time person was using node: 0 days 00:24:38
Percentage of node usage per day: 28.0 




Extra activity 1 15432
0 days 04:17:12
MFALG Assembly 18104
0 days 05:01:44
Screws in Boards 15253
0 days 04:14:13
TEKOX Blue Brown Cables 325
0 days 00:05:25
TEKOX Red Black Cables 460
0 days 00:07:40


Fernando 48096
0 days 13:21:36
Luis 1478
0 days 00:24:38


c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

